In [16]:
!pip install requests beautifulsoup4 openpyxl


In [17]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


#Aajtak



In [18]:
# ✅ All major Aaj Tak categories to scrape
base_url = "https://www.aajtak.in"
categories = [
    "india-news",           # भारत
    "world-news",           # विश्व
    "sports-news",          # खेल
    "entertainment-news",   # मनोरंजन
    "business-news",        # बिजनेस
    "technology-news",      # टेक्नोलॉजी
    "religion-news",        # धर्म
    "crime-news",           # जुर्म
    "lifestyle-news",       # लाइफस्टाइल
    "auto-news",            # ऑटो
    "education-news",       # एजुकेशन
    "science-news",         # साइंस न्यूज़
    "health-news",          # हेल्थ न्यूज़
    "trending-news",        # ट्रेंडिंग
    "agriculture-news",     # कृषि समाचार
    "fact-check",           # फैक्ट चेक
    "jokes-news",           # चुटकुले
    "literature-news",      # साहित्य
    "rajasthan-news",       # राजस्थान
    "madhya-pradesh-news",  # मध्य प्रदेश
    "elections-news",       # चुनाव
    "opinion-news",         # विचार एवं विश्लेषण
    "legal-news",           # लीगल न्यूज़
    "coronavirus-news"      # कोरोना
]

# 🔁 Number of pages to scrape per category
pages_per_category = 3


In [19]:
article_links = set()

for category in categories:
    for page in range(1, pages_per_category + 1):
        url = f"{base_url}/{category}/page-{page}"
        print("Fetching page:", url)
        try:
            res = requests.get(url)
            soup = BeautifulSoup(res.content, "html.parser")
            anchors = soup.find_all("a", href=True)
            for a in anchors:
                text = a.get_text(strip=True)
                href = a['href']
                if any('\u0900' <= c <= '\u097F' for c in text):  # Hindi text
                    if href.startswith("http"):
                        article_links.add(href)
                    elif href.startswith("/"):
                        article_links.add(base_url + href)
            time.sleep(1)  # Be polite to the server
        except Exception as e:
            print(f"Error on {url}: {e}")


Fetching page: https://www.aajtak.in/india-news/page-1
Error on https://www.aajtak.in/india-news/page-1: name 'time' is not defined
Fetching page: https://www.aajtak.in/india-news/page-2
Error on https://www.aajtak.in/india-news/page-2: name 'time' is not defined
Fetching page: https://www.aajtak.in/india-news/page-3
Error on https://www.aajtak.in/india-news/page-3: name 'time' is not defined
Fetching page: https://www.aajtak.in/world-news/page-1
Error on https://www.aajtak.in/world-news/page-1: name 'time' is not defined
Fetching page: https://www.aajtak.in/world-news/page-2
Error on https://www.aajtak.in/world-news/page-2: name 'time' is not defined
Fetching page: https://www.aajtak.in/world-news/page-3
Error on https://www.aajtak.in/world-news/page-3: name 'time' is not defined
Fetching page: https://www.aajtak.in/sports-news/page-1
Error on https://www.aajtak.in/sports-news/page-1: name 'time' is not defined
Fetching page: https://www.aajtak.in/sports-news/page-2
Error on https://w

In [20]:
def extract_hindi_article(url):
    try:
        res = requests.get(url, timeout=10)
        soup = BeautifulSoup(res.content, 'html.parser')

        # Get title
        title_tag = soup.find('h1')
        title = title_tag.text.strip() if title_tag else "No Title"

        # Get paragraphs with Hindi text
        content = ""
        for p in soup.find_all('p'):
            text = p.get_text(strip=True)
            if any('\u0900' <= c <= '\u097F' for c in text):
                content += text + " "

        return {"Title": title, "Link": url, "Content": content.strip()}
    except Exception as e:
        return {"Title": "Error", "Link": url, "Content": str(e)}


In [21]:
data = []
max_articles = 500  # Set limit to avoid long runtimes

for i, link in enumerate(article_links):
    if i >= max_articles:
        break
    print(f"[{i+1}] Scraping: {link}")
    article = extract_hindi_article(link)
    if article["Content"]:
        data.append(article)
    #time.sleep(1)  # Respectful scraping


[1] Scraping: https://www.aajtak.in/topic/ai-anchors
[2] Scraping: https://www.aajtak.in/elections/assembly-chunav/maharashtra
[3] Scraping: https://www.aajtak.in/short-videos/video/talking-about-his-debut-season-in-the-ipl-virat-kohli-recalled-the-fear-and-pressure-of-sharing-the-dressing-room-with-legends-2212822-2025-04-09
[4] Scraping: https://www.aajtak.in/elections/assembly-chunav
[5] Scraping: https://www.aajtak.in/author/ai-anchors
[6] Scraping: https://www.aajtak.in/games
[7] Scraping: https://www.aajtak.in/world/video/india-imposes-significant-penalty-on-bangladesh-by-halting-transshipment-facilities-ytvd-2212864-2025-04-09
[8] Scraping: https://www.aajtak.in/topic/america-president-election
[9] Scraping: https://www.aajtak.in/fact-check
[10] Scraping: https://www.aajtak.in/madhya-pradesh
[11] Scraping: https://www.aajtak.in/education/knowledge
[12] Scraping: https://www.aajtak.in/lifestyle
[13] Scraping: https://www.aajtak.in/business/tools-and-calculators/income-tax-calcula

In [22]:
df = pd.DataFrame(data)
df.to_excel("aajtak_hindi_articles.xlsx", index=False, engine='openpyxl')


In [23]:
df.shape

(117, 3)

In [24]:
import time

In [25]:
base_url = "https://www.bbc.com"
topic_urls = [
    "https://www.bbc.com/hindi/topics/cxnyy8ll5v0t",   # भारत
    "https://www.bbc.com/hindi/topics/c0m7zgx5x4vt",   # अंतरराष्ट्रीय
    "https://www.bbc.com/hindi/topics/c340q9kvdn7t",   # मनोरंजन
    "https://www.bbc.com/hindi/topics/cwlw57r9y92t",   # विज्ञान
    "https://www.bbc.com/hindi/topics/c2lej05epw2t",   # खेल
    "https://www.bbc.com/hindi/topics/cvjp2jr0kzdt",   # फैक्ट चेक
]


In [26]:
def get_article_links_from_topic(topic_url, max_articles=100):
    links = set()
    res = requests.get(topic_url)
    soup = BeautifulSoup(res.content, "html.parser")
    anchors = soup.find_all("a", href=True)

    for a in anchors:
        href = a["href"]
        if href.startswith("/hindi/") and len(href.split("/")) > 3:
            full_url = base_url + href
            links.add(full_url)
        if len(links) >= max_articles:
            break
    return links


In [27]:
def extract_bbc_hindi_article(url):
    try:
        res = requests.get(url, timeout=10)
        soup = BeautifulSoup(res.content, 'html.parser')

        title_tag = soup.find('h1')
        title = title_tag.get_text(strip=True) if title_tag else "No Title"

        content = ""
        for p in soup.find_all('p'):
            text = p.get_text(strip=True)
            if any('\u0900' <= c <= '\u097F' for c in text):  # Hindi detection
                content += text + " "

        return {
            "Title": title,
            "Link": url,
            "Content": content.strip()
        }
    except Exception as e:
        return {
            "Title": "Error",
            "Link": url,
            "Content": str(e)
        }


In [28]:
all_articles = []

for topic_url in topic_urls:
    print(f"Scraping topic: {topic_url}")
    links = get_article_links_from_topic(topic_url, max_articles=500)

    for i, link in enumerate(links):
        print(f"  ({i+1}) Scraping: {link}")
        data = extract_bbc_hindi_article(link)
        if data["Content"]:
            all_articles.append(data)
        time.sleep(1)


Scraping topic: https://www.bbc.com/hindi/topics/cxnyy8ll5v0t
  (1) Scraping: https://www.bbc.com/hindi/topics/c2e4q0z9qznt
  (2) Scraping: https://www.bbc.com/hindi/topics/cv22x3pmrvmt
  (3) Scraping: https://www.bbc.com/hindi/topics/c2lej0594knt
  (4) Scraping: https://www.bbc.com/hindi/topics/c9wpm0en87xt
  (5) Scraping: https://www.bbc.com/hindi/topics/ckdxnkz7607t
  (6) Scraping: https://www.bbc.com/hindi/topics/cw9kv0kpxydt
  (7) Scraping: https://www.bbc.com/hindi/topics/cr50ykdx5wqt
  (8) Scraping: https://www.bbc.com/hindi/topics/cwr9j8g1kj9t
  (9) Scraping: https://www.bbc.com/hindi/topics/c06gq3n0pp7t
Scraping topic: https://www.bbc.com/hindi/topics/c0m7zgx5x4vt
  (1) Scraping: https://www.bbc.com/hindi/topics/c2e4q0z9qznt
  (2) Scraping: https://www.bbc.com/hindi/topics/cv22x3pmrvmt
  (3) Scraping: https://www.bbc.com/hindi/topics/c2lej0594knt
  (4) Scraping: https://www.bbc.com/hindi/topics/c9wpm0en87xt
  (5) Scraping: https://www.bbc.com/hindi/topics/ckdxnkz7607t
  (6) Sc

In [29]:
df = pd.DataFrame(all_articles)
df.to_excel("BBC_Hindi_All_Articles.xlsx", index=False, engine='openpyxl')

In [31]:
df.shape

(54, 3)

#NDTV Hindi

In [10]:
!pip install requests openpyxl


In [15]:
import requests
import pandas as pd

# Replace this with your actual API key
api_key = "ade36383ae334bf0ad161f299f3c940d"

# Base URL for NewsAPI
url = f"https://newsapi.org/v2/everything?q=hindi&language=hi&pageSize=100&apiKey={api_key}"

# Make the request
response = requests.get(url)
data = response.json()

# Check if the response is successful
if data.get("status") == "ok":
    articles = []
    for article in data["articles"]:
        articles.append({
            "Title": article["title"],
            "Link": article["url"],
            "Content": article.get("content", "Not available")
        })

    # Save to Excel
    df = pd.DataFrame(articles)
    df.to_excel("hindi_news.xlsx", index=False)
    print("✅ Saved to hindi_news.xlsx")
else:
    print("❌ Error:", data.get("message", "Unknown error"))


✅ Saved to hindi_news.xlsx


In [16]:
df.shape

(52, 3)

In [17]:
df

,Title,Link,Content
0,तू मेरी दुआओं में शामिल है इस तरह... अपनों को ...,https://www.aajtak.in/india/news/story/eid-202...,"Eid Wishes in hindi: , . . - . . - . . .\r..."
1,"'पिचकारी से बरसे सिर्फ प्यार, ऐसे मनाना होली क...",https://www.aajtak.in/india/news/story/happy-h...,Copyright © 2025 Living Media India Limited. F...
2,मुल्लांपुर स्टेडियम की पिच कैसी रहेगी? जानें ह...,https://navbharattimes.indiatimes.com/sports/c...,": 18 ( ) 180 50 , ? 10 - PBKS vs CSK , PBKS vs..."
3,कोलकाता स्टेडियम की पिच कैसी रहेगी? जानें हेड-...,https://navbharattimes.indiatimes.com/sports/c...,": () () - - - , , : 44 , 14 , , 2025 12 27 , ,..."
4,"जूता खरीदने के नहीं थे पैसे, भाई-पिता की मौत स...",https://navbharattimes.indiatimes.com/sports/c...,": , -19 , - , - , , , , - , - U16 , MRF U19 6 ..."
5,मेरा दिल धड़क रहा था... नीता अंबानी ने बताया म...,https://navbharattimes.indiatimes.com/sports/c...,": () () 8 (WPL) 44 66 7 149 9 141 , - ... ! , ..."
6,जरूरी खबर : द्वारका एक्सप्रेसवे एलिवेटेड सर्वि...,https://navbharattimes.indiatimes.com/state/pu...,": 18 30 (NHAI) 31 , \r\n , , , \r\n , 31 , \r\..."
7,"वानखेड़े स्टेडियम की पिच, हेड टू हेड, मौसम... ...",https://navbharattimes.indiatimes.com/sports/c...,Mumbai Indians vs Kolkata Knight Riders Picth ...
8,भाषा विवाद के बीच आंध्र के सीएम नायडू का झलका ...,https://navbharattimes.indiatimes.com/state/an...,"<ul><li> , ' ' ? </li><li></li><li>61 , </li><..."
9,गुरुग्राम और NIT जानेवालों को मिलेगी बड़ी राहत...,https://navbharattimes.indiatimes.com/state/pu...,"<ul><li> , VIDEO</li><li></li><li>Adv: , ? .. ..."


#Bhaskar

In [39]:
# Step 1: Install required libraries
!pip install requests beautifulsoup4 pandas openpyxl

# Step 2: Import libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Step 3: Define URL and headers
url = "https://www.bhaskar.com/"  # Homepage or Hindi news section URL
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}

# Step 4: Send HTTP request
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')

# Step 5: Find article links on the homepage
# Note: Update the class/tag based on the structure of the site
articles = soup.find_all('a', href=True)
article_links = [a['href'] for a in articles if '/national/' in a['href'] or '/latest-news/' in a['href']]

# Optional: Remove duplicates and limit to a few articles
article_links = list(set(article_links))[:500]

# Step 6: Extract title and content from each article
news_data = []

for link in article_links:
    # Handle relative URLs
    if not link.startswith("http"):
        link = "https://www.bhaskar.com" + link

    try:
        article_resp = requests.get(link, headers=headers)
        article_soup = BeautifulSoup(article_resp.content, 'html.parser')

        # Extract title and content — update tags/classes as needed
        title = article_soup.find('h1').text.strip()
        paragraphs = article_soup.find_all('p')
        content = "\n".join([p.text.strip() for p in paragraphs if p.text.strip() != ""])

        news_data.append({
            'Title': title,
            'Link': link,
            'Content': content
        })
    except Exception as e:
        print(f"Failed to scrape {link}: {e}")

# Step 7: Save data to Excel
df = pd.DataFrame(news_data)
df.to_excel("bhaskar_hindi_news_articles.xlsx", index=False)

print("✅ News articles saved to bhaskar_hindi_news_articles.xlsx")


✅ News articles saved to bhaskar_hindi_news_articles.xlsx


In [40]:
df.shape

(13, 3)

#LiveHindustan


In [43]:
# Step 1: Install required libraries
!pip install requests beautifulsoup4 pandas openpyxl

# Step 2: Import libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Step 3: Define base URL and headers
base_url = "https://www.livehindustan.com"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}

# Step 4: Get homepage or latest news page
url = "https://www.livehindustan.com/national"
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, "html.parser")

# Step 5: Extract article links from homepage
article_tags = soup.find_all("a", href=True)
article_links = []

for tag in article_tags:
    link = tag["href"]
    if "/national/" in link and ".html" in link:
        full_link = base_url + link if not link.startswith("http") else link
        article_links.append(full_link)

# Remove duplicates and limit to 10 articles
article_links = list(set(article_links))[:500]

# Step 6: Scrape title and content from each article
news_data = []

for link in article_links:
    try:
        article_resp = requests.get(link, headers=headers)
        article_soup = BeautifulSoup(article_resp.content, "html.parser")

        # Extract title
        title_tag = article_soup.find("h1")
        title = title_tag.text.strip() if title_tag else "No title"

        # Extract paragraphs from article content
        content_div = article_soup.find("div", class_="article-cont")
        if content_div:
            paragraphs = content_div.find_all("p")
            content = "\n".join([p.text.strip() for p in paragraphs if p.text.strip()])
        else:
            content = "No content found"

        news_data.append({
            "Title": title,
            "Link": link,
            "Content": content
        })

    except Exception as e:
        print(f"❌ Failed to scrape {link}: {e}")

# Step 7: Save to Excel
df = pd.DataFrame(news_data)
df.to_excel("livehindustan_hindi_news.xlsx", index=False)

print("✅ News articles saved to livehindustan_hindi_news.xlsx")


✅ News articles saved to livehindustan_hindi_news.xlsx


In [44]:
df.shape

(18, 3)

In [56]:
# Install dependencies
!pip install requests pandas openpyxl

import requests
import pandas as pd
import time

api_key = "pub_7940577dcd1a5524534b467fbbdd14322e585"  # <- Replace this with your actual NewsData.io API key
base_url = "https://newsdata.io/api/1/news"

# Parameters: Hindi language, India, general news
params = {
    "apikey": api_key,
    "language": "hi",
    "country": "in",
    "category": "top",
    "page": 1
}

all_articles = []

for page in range(1, 21):  # 20 pages * ~50 = ~1000 articles
    params["page"] = page
    response = requests.get(base_url, params=params)
    data = response.json()

    results = data.get("results", [])
    if not results:
        break

    for item in results:
        all_articles.append({
            "Title": item.get("title", ""),
            "Link": item.get("link", ""),
            "Content": item.get("description", "") or ""
        })

    print(f"✅ Page {page}: Collected {len(results)} articles")
    time.sleep(1)

# Save to Excel
df = pd.DataFrame(all_articles)
df.to_excel("hindi_news_articles_1000.xlsx", index=False)
print(f"\n✅ DONE: Saved {len(df)} articles to hindi_news_articles_1000.xlsx")


AttributeError: 'str' object has no attribute 'get'

In [57]:
# Install dependencies
!pip install requests pandas openpyxl

import requests
import pandas as pd
import time

api_key = "pub_7940577dcd1a5524534b467fbbdd14322e585"  # <- Replace this with your actual NewsData.io API key
base_url = "https://newsdata.io/api/1/news"

# Parameters: Hindi language, India, general news
params = {
    "apikey": api_key,
    "language": "hi",
    "country": "in",
    "category": "top",
    "page": 1
}

all_articles = []

for page in range(1, 21):  # 20 pages * ~50 = ~1000 articles
    params["page"] = page
    response = requests.get(base_url, params=params)
    data = response.json()

    # Check if the response contains an error
    if data.get("status") == "error":
        print(f"❌ Error on page {page}: {data.get('message')}")
        # Handle the error, e.g., break the loop or skip the page
        break # Breaking loop in case of error

    results = data.get("results", [])
    if not results:
        break

    for item in results:
        all_articles.append({
            "Title": item.get("title", ""),
            "Link": item.get("link", ""),
            "Content": item.get("description", "") or ""
        })

    print(f"✅ Page {page}: Collected {len(results)} articles")
    time.sleep(1)

# Save to Excel
df = pd.DataFrame(all_articles)
df.to_excel("hindi_news_articles_1000.xlsx", index=False)
print(f"\n✅ DONE: Saved {len(df)} articles to hindi_news_articles_1000.xlsx")

❌ Error on page 1: None

✅ DONE: Saved 0 articles to hindi_news_articles_1000.xlsx


In [58]:
# STEP 1: Install dependencies
!pip install requests pandas openpyxl

# STEP 2: Import libraries
import requests
import pandas as pd
import time

# STEP 3: Set your API key here
api_key = "pub_7940577dcd1a5524534b467fbbdd14322e585"  # 🔁 Replace with your actual NewsData.io API key
base_url = "https://newsdata.io/api/1/news"

# STEP 4: Base query parameters
params = {
    "apikey": api_key,
    "language": "hi",
    "country": "in",
    "category": "top",
    "page": 1
}

# STEP 5: Loop to collect up to 1000+ articles
all_articles = []

for page in range(1, 21):  # 20 pages * ~50 = ~1000 articles
    params["page"] = page
    response = requests.get(base_url, params=params)

    try:
        data = response.json()
    except Exception as e:
        print(f"❌ Failed to parse JSON on page {page}: {e}")
        continue

    results = data.get("results", [])

    if not isinstance(results, list):
        print(f"⚠️ Unexpected results format on page {page}: {results}")
        continue

    for item in results:
        if isinstance(item, dict):
            all_articles.append({
                "Title": item.get("title", ""),
                "Link": item.get("link", ""),
                "Content": item.get("description", "") or ""
            })

    print(f"✅ Page {page}: Collected {len(results)} articles")
    time.sleep(1)  # Be polite to the API

# STEP 6: Save to Excel
df = pd.DataFrame(all_articles)
df.to_excel("hindi_news_articles_1000.xlsx", index=False)
print(f"\n🎉 DONE: Saved {len(df)} articles to 'hindi_news_articles_1000.xlsx'")


⚠️ Unexpected results format on page 1: {'message': 'The provided value for the next page is invalid. For more information on pagination, please refer to the Newsdata documentation at: https://newsdata.io/documentation/#pagination', 'code': 'UnsupportedFilter'}
⚠️ Unexpected results format on page 2: {'message': 'The provided value for the next page is invalid. For more information on pagination, please refer to the Newsdata documentation at: https://newsdata.io/documentation/#pagination', 'code': 'UnsupportedFilter'}
⚠️ Unexpected results format on page 3: {'message': 'The provided value for the next page is invalid. For more information on pagination, please refer to the Newsdata documentation at: https://newsdata.io/documentation/#pagination', 'code': 'UnsupportedFilter'}
⚠️ Unexpected results format on page 4: {'message': 'The provided value for the next page is invalid. For more information on pagination, please refer to the Newsdata documentation at: https://newsdata.io/documenta

In [59]:
import requests
import pandas as pd

# Your NewsAPI.org API key
api_key = 'ade36383ae334bf0ad161f299f3c940d'  # Replace with your actual API key

# Base URL for NewsAPI.org
base_url = 'https://newsapi.org/v2/everything'

# Parameters for the API request
params = {
    'apiKey': api_key,
    'language': 'hi',  # Hindi language
    'q': 'समाचार',     # Generic keyword for news; adjust as needed
    'pageSize': 100,   # Number of articles per page (max 100)
    'page': 1          # Starting page
}

all_articles = []

# Loop to fetch multiple pages of results
while len(all_articles) < 1000:
    response = requests.get(base_url, params=params)
    data = response.json()

    if response.status_code != 200 or 'articles' not in data:
        print(f"Error: {data.get('message', 'Failed to fetch data')}")
        break

    articles = data['articles']
    all_articles.extend(articles)

    # Check if there are more articles to fetch
    if len(articles) < params['pageSize']:
        break  # No more articles available
    else:
        params['page'] += 1  # Move to the next page

# Create a DataFrame from the collected articles
df = pd.DataFrame(all_articles)

# Save the DataFrame to an Excel file
df.to_excel('Hindi_News_Articles.xlsx', index=False)

print(f"Fetched {len(df)} articles and saved to 'Hindi_News_Articles.xlsx'")


Error: You have requested too many results. Developer accounts are limited to a max of 100 results. You are trying to request results 100 to 200. Please upgrade to a paid plan if you need more results.
Fetched 100 articles and saved to 'Hindi_News_Articles.xlsx'


In [60]:
df.shape

(100, 8)

In [61]:
df

,source,author,title,description,url,urlToImage,publishedAt,content
0,"{'id': None, 'name': 'Aajtak.in'}",aajtak.in,Aaj Ki Taza Khabar: पढ़ें 03 मार्च 2025 की शाम ...,"आज शाम की ताजा खबर (Aaj Ki Taza Khabar), 03 मा...",https://www.aajtak.in/india/news/story/aaj-ki-...,https://akm-img-a-in.tosshub.com/aajtak/images...,2025-04-03T12:27:10Z,"(Aaj Ki Taza Khabar), 03 2025 : . , . - , . (r..."
1,"{'id': None, 'name': 'Aajtak.in'}",aajtak.in,पढ़ें 05 अप्रैल 2025 की सुबह की टॉप खबरें और अन...,"आज की ताजा खबर (Aaj Ki Taza Khabar), 05 अप्रैल...",https://www.aajtak.in/india/news/story/aaj-ki-...,https://akm-img-a-in.tosshub.com/aajtak/images...,2025-04-05T03:28:39Z,". . . BJP . JDU . , . 300 500 . ...\r\nPM Mod..."
2,"{'id': None, 'name': 'Aajtak.in'}",aajtak.in,Aaj Ki Taza Khabar: पढ़ें 10 मार्च 2025 की सुबह...,बैंक ऑफ कनाडा और बैंक ऑफ इंग्लैंड के पूर्व गवर...,https://www.aajtak.in/india/news/story/aaj-ki-...,https://akm-img-a-in.tosshub.com/aajtak/images...,2025-03-10T04:24:03Z,". (ED) . . . . . , . . ' '. 5 ...\r\n , \r\n ...."
3,"{'id': None, 'name': 'Aajtak.in'}",aajtak.in,Aaj Ki Taza Khabar: पढ़ें 17 मार्च 2025 की सुबह...,प्रधानमंत्री नरेंद्र मोदी ने रविवार को अमेरिकी...,https://www.aajtak.in/india/news/story/aaj-ki-...,https://akm-img-a-in.tosshub.com/aajtak/images...,2025-03-17T04:03:10Z,"AI (Lex Fridman) 17 . , , (RSS) . , 96 . . 21..."
4,"{'id': None, 'name': 'Aajtak.in'}",aajtak.in,Aaj Ki Taza Khabar: पढ़ें 13 मार्च 2025 की शाम ...,मशकफ टनल में खड़ी ट्रेन को लेकर पाकिस्तानी सेन...,https://www.aajtak.in/india/news/story/aaj-ki-...,https://akm-img-a-in.tosshub.com/aajtak/images...,2025-03-13T13:39:41Z,". (BLA) . 2025-26 . '' '' . 5 ...\r\n , ... BL..."
...,...,...,...,...,...,...,...,...
95,"{'id': None, 'name': 'Moneycontrol'}",Moneycontrol Hindi,"Thailand, Myanmar Earthquake: भूकंप ने छीनी 14...","Thailand, Myanmar Earthquake : भारत के पड़ोसी ...",https://hindi.moneycontrol.com/world/thailand-...,https://images.moneycontrol.com/static-hindine...,2025-03-28T17:10:06Z,"Thailand, Myanmar Earthquake : 7.7 144 732 , \..."
96,"{'id': None, 'name': 'Moneycontrol'}",Moneycontrol Hindi,EPFO मेंबर्स के लिए खुशखबरी! ₹5 लाख तक बढ़ेगी ...,EPFO Update: अगर आप नौकरीपेशा हैं और EPFO में ...,https://hindi.moneycontrol.com/news/your-money...,https://images.moneycontrol.com/static-hindine...,2025-03-31T14:32:55Z,"EPFO Update: EPFO PF , ! EPFO PF (ASAC) 1 5 , ..."
97,"{'id': None, 'name': 'Moneycontrol'}",Moneycontrol Hindi,Ration Card e-KYC: 5 लाख लाभार्थियों की ई-केवा...,केंद्र सरकार ने राशन कार्डधारकों के लिए ई-केवा...,https://hindi.moneycontrol.com/news/your-money...,https://images.moneycontrol.com/static-hindine...,2025-04-04T02:44:20Z,"- - , 30 2025 - 31 , - , - , \r\n - \r\n5 - \r..."
98,"{'id': None, 'name': 'Moneycontrol'}",Moneycontrol Hindi,Trump on Rate Cut: 'कहां है महंगाई?' ट्रंप बोल...,Trump on Rate Cut: अमेरिकी राष्ट्रपति डोनाल्ड ...,https://hindi.moneycontrol.com/world/trump-us-...,https://images.moneycontrol.com/static-hindine...,2025-04-07T15:17:08Z,"Trump on Rate Cut: , "" "" (FOMC) 6-7 , ( ) \r\n..."


In [64]:
# prompt: update the dataframe to contain only the title,url and content and change heading of column url to link

import pandas as pd

# Assuming your DataFrame is named 'df'
df = df[['title', 'url', 'content']]
df = df.rename(columns={'Link': 'url'})


In [65]:
df.shape


(100, 3)

In [66]:
df.to_excel('Hindi_News_Articles.xlsx', index=False)


In [68]:
import requests
import pandas as pd
import time

api_key = 'ade36383ae334bf0ad161f299f3c940d'

base_url = 'https://newsapi.org/v2/everything'
keywords = ['समाचार', 'ताज़ा खबर', 'ब्रेकिंग न्यूज़', 'आज की खबरें',  # General
    'चुनाव', 'प्रधानमंत्री', 'सरकार', 'संसद', 'विपक्ष', 'भाजपा', 'कांग्रेस',  # Politics
    'शेयर बाजार', 'बजट', 'आर्थिक संकट', 'जीएसटी', 'व्यापार',  # Business
    'बॉलीवुड', 'फिल्म', 'सिनेमा', 'अभिनेता', 'अभिनेत्री', 'वेब सीरीज़',  # Entertainment
    'क्रिकेट', 'आईपीएल', 'विश्व कप', 'फुटबॉल', 'हॉकी', 'ओलंपिक',  # Sports
    'अमेरिका', 'रूस', 'चीन', 'पाकिस्तान', 'संयुक्त राष्ट्र',  # International
    'मोबाइल', 'इंटरनेट', 'आर्टिफिशियल इंटेलिजेंस', 'साइबर सुरक्षा', 'तकनीकी समाचार',  # Tech
    'योग', 'आयुर्वेद', 'स्वास्थ्य', 'कोरोना वायरस', 'टीकाकरण', 'फिटनेस',  # Health
    'जलवायु परिवर्तन', 'प्रदूषण', 'वनों की कटाई', 'पर्यावरण संरक्षण',  # Environment
    'बोर्ड परीक्षा', 'प्रवेश परीक्षा', 'छात्रवृत्ति', 'स्कूल', 'विश्वविद्यालय']  # Add more relevant keywords

all_articles = []

for keyword in keywords:
    params = {
        'apiKey': api_key,
        'language': 'hi',
        'q': keyword,
        'pageSize': 100,
        'page': 1
    }

    response = requests.get(base_url, params=params)
    data = response.json()

    if 'articles' in data:
        all_articles.extend(data['articles'])

    print(f"Fetched {len(data.get('articles', []))} for keyword '{keyword}'")

    # Be nice to API
    time.sleep(2)

df = pd.DataFrame(all_articles).drop_duplicates(subset=['title'])

df.to_excel('Hindi_News_Articles_Multiple_Keywords.xlsx', index=False)
print(f"Total unique articles fetched: {len(df)}")


Fetched 100 for keyword 'समाचार'
Fetched 0 for keyword 'ताज़ा खबर'
Fetched 0 for keyword 'ब्रेकिंग न्यूज़'
Fetched 62 for keyword 'आज की खबरें'
Fetched 100 for keyword 'चुनाव'
Fetched 100 for keyword 'प्रधानमंत्री'
Fetched 100 for keyword 'सरकार'
Fetched 100 for keyword 'संसद'
Fetched 100 for keyword 'विपक्ष'
Fetched 100 for keyword 'भाजपा'
Fetched 100 for keyword 'कांग्रेस'
Fetched 100 for keyword 'शेयर बाजार'
Fetched 100 for keyword 'बजट'
Fetched 40 for keyword 'आर्थिक संकट'
Fetched 67 for keyword 'जीएसटी'
Fetched 100 for keyword 'व्यापार'
Fetched 100 for keyword 'बॉलीवुड'
Fetched 100 for keyword 'फिल्म'
Fetched 71 for keyword 'सिनेमा'
Fetched 100 for keyword 'अभिनेता'
Fetched 75 for keyword 'अभिनेत्री'
Fetched 3 for keyword 'वेब सीरीज़'
Fetched 100 for keyword 'क्रिकेट'
Fetched 100 for keyword 'आईपीएल'
Fetched 34 for keyword 'विश्व कप'
Fetched 14 for keyword 'फुटबॉल'
Fetched 22 for keyword 'हॉकी'
Fetched 38 for keyword 'ओलंपिक'
Fetched 100 for keyword 'अमेरिका'
Fetched 100 for keywo

In [69]:
df.shape

(2981, 8)

In [70]:
# Assuming your DataFrame is named 'df'
df = df[['title', 'url', 'content']]
df = df.rename(columns={'Link': 'url'})

In [71]:
df.shape

(2981, 3)

In [76]:
df=pd.read_excel("Hindi_News_Articles_Multiple_Keywords.xlsx")

In [77]:
df.shape

(2981, 8)

In [78]:
df = df[['title', 'url', 'content']]
df = df.rename(columns={'Link': 'url'})

In [79]:
df.shape

(2981, 3)

In [80]:
df.to_excel('Hindi_News_Articles_Multiple_Keywords.xlsx', index=False)
